In [1]:
import json
import numpy as np

In [2]:
from bingo import Card, Bingo

In [3]:
with open('SGRS2020_Task25_data.json') as fh:
    data = json.load(fh)

In [4]:
game = Bingo(data)
game

<Bingo ['Alice', 'Bob', 'Colby', 'Dylan', 'Emily', 'Faith', 'Grace', 'Harris', 'Isla', 'Jack', 'Kai', 'Logan', 'Max', 'Nathan', 'Olivia', 'Poppy', 'Quinn', 'Rory', 'Sophie', 'Thomas']>

In [5]:
game.cards

{'Alice': <Card:'Alice' (2, 3, 10, 12, 15, 19, 23, 24, 25, 30, 34, 37, 41, 43, 47, 48, 53, 56, 57, 61, 64, 68, 72, 73)>,
 'Bob': <Card:'Bob' (1, 2, 6, 12, 15, 17, 18, 21, 23, 24, 33, 37, 39, 40, 48, 49, 50, 52, 53, 63, 65, 66, 67, 68)>,
 'Colby': <Card:'Colby' (3, 4, 7, 9, 10, 19, 23, 25, 28, 29, 34, 35, 42, 45, 47, 48, 49, 50, 53, 61, 63, 67, 71, 75)>,
 'Dylan': <Card:'Dylan' (1, 3, 6, 8, 12, 22, 26, 27, 28, 29, 34, 36, 41, 42, 54, 55, 56, 57, 58, 68, 70, 71, 74, 75)>,
 'Emily': <Card:'Emily' (1, 2, 3, 8, 12, 22, 23, 24, 27, 29, 35, 41, 44, 45, 51, 52, 53, 55, 58, 61, 66, 72, 74, 75)>,
 'Faith': <Card:'Faith' (4, 5, 11, 13, 14, 17, 25, 27, 28, 30, 33, 36, 38, 41, 48, 49, 56, 57, 60, 62, 63, 65, 66, 67)>,
 'Grace': <Card:'Grace' (1, 3, 6, 10, 12, 25, 26, 27, 29, 30, 32, 38, 43, 44, 48, 55, 56, 58, 59, 64, 68, 69, 73, 75)>,
 'Harris': <Card:'Harris' (1, 4, 5, 7, 8, 19, 26, 27, 28, 29, 34, 35, 42, 45, 54, 55, 56, 57, 58, 61, 62, 67, 70, 73)>,
 'Isla': <Card:'Isla' (2, 3, 8, 12, 15, 17, 1

In [6]:
game.cards['Alice'].columns(tagged=True)

{'A.C0': (2, 3, 10, 12, 15),
 'A.C1': (19, 23, 24, 25, 30),
 'A.C3': (47, 48, 53, 56, 57),
 'A.C4': (61, 64, 68, 72, 73)}

In [7]:
game.cards['Alice'].specials(creator=np.array)

{'A.D1S': array([ 2, 23,  0, 56, 73]),
 'A.D2S': array([15, 25,  0, 48, 61]),
 'A.R2S': array([10, 24,  0, 53, 68]),
 'A.C2S': array([34, 37,  0, 41, 43])}

In [8]:
game.cards['Alice'].solutions(tagged=False, creator=set)

({2, 19, 34, 47, 61},
 {3, 23, 37, 48, 64},
 {12, 25, 41, 56, 72},
 {15, 30, 43, 57, 73},
 {2, 19, 34, 47, 61},
 {3, 23, 37, 48, 64},
 {12, 25, 41, 56, 72},
 {15, 30, 43, 57, 73},
 {0, 2, 23, 56, 73},
 {0, 15, 25, 48, 61},
 {0, 10, 24, 53, 68},
 {0, 34, 37, 41, 43})